In [ ]:
%matplotlib inline
import numpy as np
import numpy.ma as ma
import matplotlib.pyplot as plt
import math

In [ ]:
from astropy.table import Table
t = Table.read('../../ijoncour/StandCat/Tgas200_Wright2003_2MASS_FinCat.vot', format='votable')

In [ ]:
print("The astropy reader loads the data into the following variable names:")
print(t.colnames)

In [ ]:
#Rename columns
Source = t["Source"]
Gmag = t["__Gmag_"]
Vmag = t["VTmag"]
Bmag = t["BTmag"]
Jmag = t["Jmag"]
Hmag = t["Hmag"]
Kmag = t["Kmag"]
QFlags = t["Qfl"]
DistW = t["Dist"]
s1= t["d_arcsec"]
s2 = t["d_arcsec_2"]
plx = t['Plx']
Teff = t['Teff']
e_Jmag = t['e_Jmag']
e_Kmag = t['e_Kmag']
e_Hmag = t['e_Hmag']
SpType = t["SpType"]
dup = t['Dup']
RA = t["RAJ2000"]
DEC = t["_DEJ2000"]

In [ ]:
#calculate distance and absolute magnitudes and plot color index vs. absolute g magnitude for different luminosity classes
Dist = 1000./t["Plx"]
Vmags = t['VTmag'] - 5 * np.log10(Dist) + 5
Bmags = t['BTmag'] - 5 * np.log10(Dist) + 5
Gmags = t['__Gmag_'] - 5 * np.log10(Dist) + 5
Jmags = t['Jmag'] -5 * np.log10(Dist) + 5
Hmags = t['Hmag'] - 5 * np.log10(Dist) + 5
Kmags = t['Kmag'] - 5 * np.log10(Dist) + 5

In [ ]:
#mask all values with dup = 1
imask = 0
for i in range(0,len(dup)):
    if (dup[i] > 0.5):
        imask = imask + 1
        row = t[i]
        idx = row._index
        cols = row._table.columns.values()
        for col in cols:
            col.mask[i] = True
for i in range(0,len(dup)):
    if(dup[i] > 0.5):
        print(i,dup[i])
print(" Masked ",imask," rows in which dup is equal to 1")

In [ ]:
#look at number of duplicate matches
Test = Source.compressed()
unique = set()
Gdups = set()

for i in range(0,len(Test)):
    if(Test[i] in unique):
        print("GAIA Duplicate Source: ",Test[i])
        Gdups.add(Test[i])
    else:
        unique.add(Test[i])
        last = Test[i]
print("Finished GAIA Duplicate Check. Found: ", len(Gdups))

for i in range(0,len(Source)):
    if(Source.mask[i] == False):
        Name = Source[i]
        if(Name in Gdups):
            row = t[i]
            idx = row._index
            cols = row._table.columns.values()
            for col in cols:
                col.mask[i] = True
print("Finished masking all GAIA duplicates. All sources involved")
TwoM = t["_2MASS"]
unique = set()
Tdups = set()
Test = TwoM.compressed()
for i in range(0,len(Test)):
    if(Test[i] in unique):
        print("2MASS Duplicate Source: ",Test[i])
        Tdups.add(Test[i])
    else:
        unique.add(Test[i])
        last = Test[i]
print("Finished 2MASS Duplicate Check. Found: ", len(Tdups))

for i in range(0,len(Source)):
    if(Source.mask[i] == False):
        Name = Source[i]
        if(Name in Tdups):
            print(Source[i],s1[i],s2[i],Gmag[i],Vmag[i]-Gmag[i],Jmag[i],Kmag[i],TwoM[i])

In [ ]:
J_G = Jmag-Gmag
plt.scatter(s1,J_G,s=2.0)
plt.show()
JGmean = np.mean(J_G)
JGstd = np.std(J_G)
print(JGmean)
print(JGstd)

In [ ]:
#masked rows with J-G greater than 5 sigma from mean
imask = 0
for i in range(0,len(J_G)):
    if(abs(J_G[i] - JGmean) > 5*JGstd):
        imask = imask + 1
        row = t[i]
        idx = row._index
        cols = row._table.columns.values()
        for col in cols:
            col.mask[i] = True
print(" Masked ",imask," rows based on J-G color > 5 sigma from mean")
J_G = Jmag - Gmag
plt.xlabel("Distance of 2MASS to GAIA")
plt.ylabel(" J - G magnitude")
plt.scatter(s1,J_G,s=2.0)
plt.show()
print("New sigma for J-G: ",np.std(J_G))

In [ ]:
#Translate Spectra Type luminosity class into a number from 1 to 5
#or a negative number if it is a binary or unknown spectral type.
#Note that many stars do not have a luminosity class. There are given 5.1.

SpTypeN = [0.0]*len(SpType)
for i in range(0,len(SpType)):
    if(SpType.mask[i] == False):
        SpT = SpType[i].decode()
        if(len(SpT) > 2):
            for j in range(0,len(SpT)):
                if(SpT[j] == 'I'):
                    SpTypeN[i] = SpTypeN[i] + 1
                if(SpT[j] == 'V'):
                    SpTypeN[i] = SpTypeN[i] + 5
                if(SpT[j] == '+'):
                    SpTypeN[i] = -20.
                
        else:
            SpTypeN[i] = 5.1
        
        if(SpTypeN[i] == 6):
            SpTypeN[i] = 4.
        if(SpTypeN[i] == 11):
            SpTypeN[i] = 4.5
        if(SpTypeN[i] == 9):
            SpTypeN[i] = 3.5
        if(SpTypeN[i] > 5.2):
            SpTypeN[i] = -30.
print("Finished Luminosity Class Translation")
count = 0
for i in range(0,len(SpType)):
    if(SpTypeN[i] < 0.):
        count = count + 1
test = SpType.compressed()
good = len(test) - count
print("  Number of single Stars: ",good,"  Binaries or Unknown luminosity class: ",count)

In [ ]:
#Split class V stars into different temperature ranges and find avg, min, max, and standard deviation of each range
temps = [3500,4000,4500,5000,5500,6000,6500,7000,7500,8000,8500,9000,9500,10500]
num_stars = np.zeros(len(temps))
temp_cat = np.zeros(len(Teff))
tavg = []
tmin = []
tmax = []
t_stdev = []
j = 0
for t1 in temps: 
    stars = 0
    t2 = t1 + 500
    temp_array = []
    for i in range(len(Teff)):
        if (SpTypeN[i] > 4.9 and SpTypeN[i] < 5.05):
            if (Teff[i] >= t1 and Teff[i] < t2):
                temp_cat[i] = t1
                temp_array = np.append(temp_array, Teff[i])
                stars = stars+1
    tavg = np.append(tavg,np.mean(temp_array))
    tmax = np.append(tmax,np.max(temp_array,axis =0))
    tmin = np.append(tmin,np.min(temp_array,axis =0))
    t_stdev = np.append(t_stdev,np.std(temp_array))
    print(str(t1) + 'K : ' + str(stars))
    num_stars[j] = stars
    j = j+1
print(tavg)
print(tmax)
print(tmin)
print(t_stdev)
print(num_stars)

In [ ]:
#Split Class V stars into different distance ranges, assign each star a distance category
distances = [100,150,200]
distance_cat = np.zeros(len(Dist))
stars = 0
for i in range(len(distances)): 
    if (i==0):
        d1 = 0
        d2 = distances[0]
    else:
        d1 = distances[i-1]
        d2 = d1+50
    for j in range(len(Dist)):
        if (SpTypeN[j] > 4.9 and SpTypeN[j] < 5.05):
            if (Dist[j] > d1 and Dist[j] <= d2):
                distance_cat[j] = d2
                stars = stars+1
print(stars)

In [ ]:
#Relative SED for all stars
temps2 = [4000,4500,5000,5500,6000,6500,7000,7500,8000,8500,9000,9500,10500]
wvl = [420, 532, 1235, 1662, 2159]
wvl2 = [420, 532,673, 1235, 1662, 2159]
avg_temps = np.array([])
err_temps = np.array([])
stars = np.array([])
for t in temps2: 
    G_V_array = ma.array([])
    G_B_array = ma.array([])
    G_J_array = ma.array([])
    G_H_array = ma.array([])
    G_K_array = ma.array([])
    B = ma.array([])
    V = ma.array([])
    G = ma.array([])
    J = ma.array([])
    H = ma.array([])
    K = ma.array([])
    temperatures = np.array([])
    stars = 0
    for i in range(len(temp_cat)):
        if (temp_cat[i] == t and distance_cat[i] == 100):
            G_V_array = np.append(G_V_array,Gmags[i]-Vmags[i])
            G_B_array = np.append(G_B_array,Gmags[i]-Bmags[i])
            G_J_array = np.append(G_J_array,Gmags[i]-Jmags[i])
            G_H_array = np.append(G_H_array,Gmags[i]-Hmags[i])
            G_K_array = np.append(G_K_array,Gmags[i]-Kmags[i])
            B = np.append(B,Bmags[i])
            V = np.append(V,Vmags[i])
            G = np.append(G,Gmags[i])
            J = np.append(J,Jmags[i])
            H = np.append(H,Hmags[i])
            K = np.append(K,Kmags[i])
            temperatures = np.append(temperatures,Teff[i])
            stars = stars +1
    #average temperatures
    avg_temps = np.append(avg_temps,np.mean(temperatures))
    err_temps = np.append(err_temps,np.std(temperatures))
    #number of stars per temperature
    stars = np.append(stars,stars)
    #relative SED
    mag_array = [np.mean(G_B_array), np.mean(G_V_array), np.mean(G_J_array), np.mean(G_H_array), np.mean(G_K_array)]
    std_array = [np.std(G_B_array), np.std(G_V_array), np.std(G_J_array), np.std(G_H_array), np.std(G_K_array)]
    #absolute SED
    mag_array2 = [np.mean(B), np.mean(V), np.mean(G), np.mean(J), np.mean(H), np.mean(K)]
    std_array2 = [np.std(B), np.std(V), np.std(G), np.std(J), np.std(H), np.std(K)]
    #add standard deviation error
    #val = np.sqrt(len(G_B_array))
    #std_array = std_array/val
    #val2 = np.sqrt(len(B))
    #std_array2 = std_array2/val2
    #stack SEDs
    if (t == temps2[0]):
        temp_mags = np.array(mag_array)
        temp_std = np.array(std_array)
        abs_temp_mags = np.array(mag_array2)
        abs_temp_std = np.array(std_array2)
    else:
        temp_mags = np.vstack((temp_mags,mag_array))
        temp_std = np.vstack((temp_std,std_array))
        abs_temp_mags = np.vstack((abs_temp_mags, mag_array2))
        abs_temp_std = np.vstack((abs_temp_mags, std_array2))

In [ ]:
#Relative Magnitude SED for stars located within 100 pc of earth
temps2 = [4000,4500,5000,5500,6000,6500,7000,7500,8000,8500,9000,9500,10500]
wvl = [420, 532, 1235, 1662, 2159]
wvl2 = [420, 532,673, 1235, 1662, 2159]
avg_temps100 = np.array([])
err_temps100 = np.array([])
stars100 = np.array([])
for t in temps2: 
    G_V_array = ma.array([])
    G_B_array = ma.array([])
    G_J_array = ma.array([])
    G_H_array = ma.array([])
    G_K_array = ma.array([])
    B = ma.array([])
    V = ma.array([])
    G = ma.array([])
    J = ma.array([])
    H = ma.array([])
    K = ma.array([])
    temperatures = np.array([])
    stars = 0
    for i in range(len(temp_cat)):
        if (temp_cat[i] == t and distance_cat[i] == 100):
            G_V_array = np.append(G_V_array,Gmags[i]-Vmags[i])
            G_B_array = np.append(G_B_array,Gmags[i]-Bmags[i])
            G_J_array = np.append(G_J_array,Gmags[i]-Jmags[i])
            G_H_array = np.append(G_H_array,Gmags[i]-Hmags[i])
            G_K_array = np.append(G_K_array,Gmags[i]-Kmags[i])
            B = np.append(B,Bmags[i])
            V = np.append(V,Vmags[i])
            G = np.append(G,Gmags[i])
            J = np.append(J,Jmags[i])
            H = np.append(H,Hmags[i])
            K = np.append(K,Kmags[i])
            temperatures = np.append(temperatures,Teff[i])
            stars = stars +1
    #average temperatures
    avg_temps100 = np.append(avg_temps100,np.mean(temperatures))
    err_temps100 = np.append(err_temps100,np.std(temperatures))
    #number of stars per temperature
    stars100 = np.append(stars100,stars)
    #relative SED
    mag_array = [np.mean(G_B_array), np.mean(G_V_array), np.mean(G_J_array), np.mean(G_H_array), np.mean(G_K_array)]
    std_array = [np.std(G_B_array), np.std(G_V_array), np.std(G_J_array), np.std(G_H_array), np.std(G_K_array)]
    #absolute SED
    mag_array2 = [np.mean(B), np.mean(V), np.mean(G), np.mean(J), np.mean(H), np.mean(K)]
    std_array2 = [np.std(B), np.std(V), np.std(G), np.std(J), np.std(H), np.std(K)]
    #add standard deviation error
    val = np.sqrt(len(G_B_array))
    std_array = std_array/val
    val2 = np.sqrt(len(B))
    std_array2 = std_array2/val2
    #stack SEDs
    if (t == temps2[0]):
        temp_mags100 = np.array(mag_array)
        temp_std100 = np.array(std_array)
        abs_temp_mags100 = np.array(mag_array2)
        abs_temp_std100 = np.array(std_array2)
    else:
        temp_mags100 = np.vstack((temp_mags100,mag_array))
        temp_std100 = np.vstack((temp_std100,std_array))
        abs_temp_mags100 = np.vstack((abs_temp_mags100, mag_array2))
        abs_temp_std100 = np.vstack((abs_temp_mags100, std_array2))

In [ ]:
#Relative Magnitude SED for stars located between 100 and 150 parsecs
temps2 = [4000,4500,5000,5500,6000,6500,7000,7500,8000,8500,9000,9500,10500]
wvl = [420, 532, 1235, 1662, 2159]
wvl2 = [420, 532,673, 1235, 1662, 2159]
avg_temps150 = np.array([])
err_temps150 = np.array([])
stars150 = np.array([])
for t in temps2: 
    G_V_array = ma.array([])
    G_B_array = ma.array([])
    G_J_array = ma.array([])
    G_H_array = ma.array([])
    G_K_array = ma.array([])
    B = ma.array([])
    V = ma.array([])
    G = ma.array([])
    J = ma.array([])
    H = ma.array([])
    K = ma.array([])
    temperatures = np.array([])
    stars = 0
    for i in range(len(temp_cat)):
        if (temp_cat[i] == t and distance_cat[i] == 150):
            G_V_array = np.append(G_V_array,Gmags[i]-Vmags[i])
            G_B_array = np.append(G_B_array,Gmags[i]-Bmags[i])
            G_J_array = np.append(G_J_array,Gmags[i]-Jmags[i])
            G_H_array = np.append(G_H_array,Gmags[i]-Hmags[i])
            G_K_array = np.append(G_K_array,Gmags[i]-Kmags[i])
            B = np.append(B,Bmags[i])
            V = np.append(V,Vmags[i])
            G = np.append(G,Gmags[i])
            J = np.append(J,Jmags[i])
            H = np.append(H,Hmags[i])
            K = np.append(K,Kmags[i])
            temperatures = np.append(temperatures,Teff[i])
            stars = stars +1
    #average temperatures
    avg_temps150 = np.append(avg_temps150,np.mean(temperatures))
    err_temps150 = np.append(err_temps150,np.std(temperatures))
    #number of stars per temperature
    stars150 = np.append(stars150,stars)
    #relative SED
    mag_array = [np.mean(G_B_array), np.mean(G_V_array), np.mean(G_J_array), np.mean(G_H_array), np.mean(G_K_array)]
    std_array = [np.std(G_B_array), np.std(G_V_array), np.std(G_J_array), np.std(G_H_array), np.std(G_K_array)]
    #absolute SED
    mag_array2 = [np.mean(B), np.mean(V), np.mean(G), np.mean(J), np.mean(H), np.mean(K)]
    std_array2 = [np.std(B), np.std(V), np.std(G), np.std(J), np.std(H), np.std(K)]
    #add standard deviation error
    val = np.sqrt(len(G_B_array))
    std_array = std_array/val
    val2 = np.sqrt(len(B))
    std_array2 = std_array2/val2
    #stack SEDs
    if (t == temps2[0]):
        temp_mags150 = np.array(mag_array)
        temp_std150 = np.array(std_array)
        abs_temp_mags150 = np.array(mag_array2)
        abs_temp_std150 = np.array(std_array2)
    else:
        temp_mags150 = np.vstack((temp_mags150,mag_array))
        temp_std150 = np.vstack((temp_std150,std_array))
        abs_temp_mags150 = np.vstack((abs_temp_mags150, mag_array2))
        abs_temp_std150 = np.vstack((abs_temp_mags150, std_array2))

In [ ]:
#Relative Magnitude SED for stars located within 150 and 200 pc
temps2 = [4000,4500,5000,5500,6000,6500,7000,7500,8000,8500,9000,9500,10500]
wvl = [420, 532, 1235, 1662, 2159]
wvl2 = [420, 532,673, 1235, 1662, 2159]
avg_temps200 = np.array([])
err_temps200 = np.array([])
stars200 = np.array([])
for t in temps2: 
    G_V_array = ma.array([])
    G_B_array = ma.array([])
    G_J_array = ma.array([])
    G_H_array = ma.array([])
    G_K_array = ma.array([])
    B = ma.array([])
    V = ma.array([])
    G = ma.array([])
    J = ma.array([])
    H = ma.array([])
    K = ma.array([])
    temperatures = np.array([])
    stars = 0
    for i in range(len(temp_cat)):
        if (temp_cat[i] == t and distance_cat[i] == 200):
            G_V_array = np.append(G_V_array,Gmags[i]-Vmags[i])
            G_B_array = np.append(G_B_array,Gmags[i]-Bmags[i])
            G_J_array = np.append(G_J_array,Gmags[i]-Jmags[i])
            G_H_array = np.append(G_H_array,Gmags[i]-Hmags[i])
            G_K_array = np.append(G_K_array,Gmags[i]-Kmags[i])
            B = np.append(B,Bmags[i])
            V = np.append(V,Vmags[i])
            G = np.append(G,Gmags[i])
            J = np.append(J,Jmags[i])
            H = np.append(H,Hmags[i])
            K = np.append(K,Kmags[i])
            temperatures = np.append(temperatures,Teff[i])
            stars = stars +1
    #average temperatures
    avg_temps200 = np.append(avg_temps200,np.mean(temperatures))
    err_temps200 = np.append(err_temps200,np.std(temperatures))
    #number of stars per temperature
    stars200 = np.append(stars200,stars)
    #relative SED
    mag_array = [np.mean(G_B_array), np.mean(G_V_array), np.mean(G_J_array), np.mean(G_H_array), np.mean(G_K_array)]
    std_array = [np.std(G_B_array), np.std(G_V_array), np.std(G_J_array), np.std(G_H_array), np.std(G_K_array)]
    #absolute SED
    mag_array2 = [np.mean(B), np.mean(V), np.mean(G), np.mean(J), np.mean(H), np.mean(K)]
    std_array2 = [np.std(B), np.std(V), np.std(G), np.std(J), np.std(H), np.std(K)]
    #add standard deviation error
    val = np.sqrt(len(G_B_array))
    std_array = std_array/val
    val2 = np.sqrt(len(B))
    std_array2 = std_array2/val2
    #stack SEDs
    if (t == temps2[0]):
        temp_mags200 = np.array(mag_array)
        temp_std200 = np.array(std_array)
        abs_temp_mags200 = np.array(mag_array2)
        abs_temp_std200 = np.array(std_array2)
    else:
        temp_mags200 = np.vstack((temp_mags200,mag_array))
        temp_std200 = np.vstack((temp_std200,std_array))
        abs_temp_mags200 = np.vstack((abs_temp_mags200, mag_array2))
        abs_temp_std200 = np.vstack((abs_temp_mags200, std_array2))

In [ ]:
#Compare Average Temperatures
print(stars100)
print(avg_temps100)
print(stars150)
print(avg_temps150)
print(stars200)
print(avg_temps200)

In [ ]:
#Plot relative SED by distance 
plt.figure()
for i in range(len(temps2)): 
    plt.errorbar(wvl,temp_mags100[i,:],yerr = temp_std100[i,:],linestyle = '-', marker = 'o',label = str(temps2[i]))
    plt.title('Relative SED for 100 pc')
    plt.xlabel('Wavelength (nm)')
    plt.ylabel('G-Mag')
    plt.legend()

plt.figure()
for i in range(len(temps2)): 
    plt.errorbar(wvl,temp_mags150[i,:],yerr = temp_std150[i,:],linestyle = '-', marker = 'o',label = str(temps2[i]))
    plt.title('Relative SED for 150 pc')
    plt.xlabel('Wavelength (nm)')
    plt.ylabel('G-Mag')
    plt.legend()

plt.figure()
for i in range(len(temps2)): 
    plt.errorbar(wvl,temp_mags200[i,:],yerr = temp_std200[i,:],linestyle = '-', marker = 'o',label = str(temps2[i]))
    plt.title('Relative SED for 200 pc')
    plt.xlabel('Wavelength (nm)')
    plt.ylabel('G-Mag')
    plt.legend()


In [ ]:
#Plot absolute SED for each distance
from matplotlib.backends.backend_pdf import PdfPages

plt.figure()
for i in range(len(temps2)): 
    plt.errorbar(wvl2,abs_temp_mags100[i,:],yerr = abs_temp_std150[i,:],linestyle = '-', marker = 'o',label = str(temps2[i]))
    plt.title('Absolute SED for 100 pc')
    plt.xlabel('Wavelength (nm)')
    plt.ylabel('Average Magnitude')
    plt.legend()
    
plt.figure()
for i in range(len(temps2)): 
    plt.errorbar(wvl2,abs_temp_mags150[i,:],yerr = abs_temp_std150[i,:],linestyle = '-', marker = 'o',label = str(temps2[i]))
    plt.title('Absolute SED for 150 pc')
    plt.xlabel('Wavelength (nm)')
    plt.ylabel('Average Magnitude')
    plt.legend()

pdf1 = PdfPages('AbsoluteSED_200pc.pdf')
plt.figure()
for i in range(len(temps2)): 
    plt.errorbar(wvl2,abs_temp_mags200[i,:],yerr = abs_temp_std200[i,:],linestyle = '-', marker = 'o',label = str(temps2[i]))
    plt.title('Absolute SED for 200 pc')
    plt.xlabel('Wavelength (nm)')
    plt.ylabel('Average Magnitude')
    plt.legend()
pdf1.savefig()
pdf1.close()

In [ ]:
#Compare Templates: For each temperature category (4000 to 10500) plot Relative SED
for i in range(len(temps2)): 
    plt.figure()
    plt.errorbar(wvl,temp_mags100[i],yerr = temp_std100[i],linestyle = '-', marker = 'o',label = '100 pc')
    plt.errorbar(wvl,temp_mags150[i],yerr = temp_std150[i],linestyle = '-', marker = 'o', label = '150 pc')
    plt.errorbar(wvl,temp_mags200[i],yerr = temp_std200[i],linestyle = '-', marker = 'o', label = '200 pc')
    plt.title('Relative SED for '+ str(temps2[i]) + 'K')
    plt.xlabel('Wavelength (nm)')
    plt.ylabel('Average Magnitude Relative to G')
    plt.legend()



In [ ]:
#Compare_templates: For each temperature category (starting at 4000) plot absolute SED
pdf2 = PdfPages('AbsoluteSED_5000K')
for i in range(len(temps2)): 
    plt.figure()
    plt.errorbar(wvl2,abs_temp_mags100[i],yerr = abs_temp_std100[i],linestyle = '-', marker = 'o',label = '100 pc')
    plt.errorbar(wvl2,abs_temp_mags150[i],yerr = abs_temp_std150[i],linestyle = '-', marker = 'o', label = '150 pc')
    plt.errorbar(wvl2,abs_temp_mags200[i],yerr = abs_temp_std200[i],linestyle = '-', marker = 'o', label = '200 pc')
    plt.title('Absolute SED for '+ str(temps2[i]) + 'K')
    plt.xlabel('Wavelength (nm)')
    plt.ylabel('Average Magnitude')
    plt.legend()
    if (temps2[i]==5000): 
        pdf2.savefig()
        pdf2.close()


In [ ]:
#Create array with values of G-K for each star in the 100, 150, and 200 templates



G_K_100 = ma.array([])
G_K_150 = ma.array([])
G_K_200 = ma.array([])

for i in range(len(Gmags)):
    if (distance_cat[i] == 100):
        if (len(G_K_100) == 0):
            GK_val = ma.array([])
            GK_val = Gmags[i]-Kmags[i]
            G_K_100 = ma.array([GK_val,Teff[i],temp_cat[i],RA[i],DEC[i]])
        else: 
            GK_val = ma.array([])
            GK_val = Gmags[i]-Kmags[i]
            G_K_100 = np.vstack((G_K_100,[GK_val,Teff[i],temp_cat[i],RA[i],DEC[i]]))
            
for i in range(len(Gmags)):
    if (distance_cat[i] == 150):
        if (len(G_K_150) == 0):
            GK_val = ma.array([])
            GK_val = Gmags[i]-Kmags[i]
            G_K_150 = ma.array([GK_val,Teff[i],temp_cat[i],RA[i],DEC[i]])
        else: 
            GK_val = ma.array([])
            GK_val = Gmags[i]-Kmags[i]
            G_K_150 = np.vstack((G_K_150,[GK_val,Teff[i],temp_cat[i],RA[i],DEC[i]]))
            
for i in range(len(Gmags)):
    if (distance_cat[i] == 200):
        if (len(G_K_200) == 0):
            GK_val = ma.array([])
            GK_val = Gmags[i]-Kmags[i]
            G_K_200 = ma.array([GK_val,Teff[i],temp_cat[i],RA[i],DEC[i]])
        else: 
            GK_val = ma.array([])
            GK_val = Gmags[i]-Kmags[i]
            G_K_200 = np.vstack((G_K_200,[GK_val,Teff[i],temp_cat[i],RA[i],DEC[i]]))

In [ ]:
#create histogram for each temperature class for 100 pc
for i in temps2:
    same_temp = np.array([])
    for j in range(len(G_K_100)):
        if (G_K_100[j,2] == i):
            same_temp = np.append(same_temp,G_K_100[j,0])
    plt.figure()
    plt.hist(same_temp,range = [min(same_temp),max(same_temp)],bins=20)
    plt.title(str(i))
    plt.show()

        

In [ ]:
for i in temps2:
    same_temp = np.array([])
    for j in range(len(G_K_150)):
        if (G_K_150[j,2] == i):
            same_temp = np.append(same_temp,G_K_150[j,0])
    plt.figure()
    plt.hist(same_temp,range = [min(same_temp),max(same_temp)], bins = 20)
    plt.title(str(i))
    plt.show()

In [ ]:
for i in temps2:
    same_temp = np.array([])
    for j in range(len(G_K_200)):
        if (G_K_200[j,2] == i):
            same_temp = np.append(same_temp,G_K_200[j,0])
    plt.figure()
    plt.hist(same_temp,range = [min(same_temp),max(same_temp)],bins = 40)
    plt.title(str(i))
    plt.show()

In [ ]:
#Plot Coordinates of all stars at 5000 K and 200 pc 
number = 0
plt.figure()
for i in range(len(G_K_200)):
    if (G_K_200[i,2] == 5500):
        number = number +1
        plt.scatter(G_K_200[i,3],G_K_200[i,4])
plt.xlabel('RA')
plt.ylabel('Dec')
plt.show()
print(number)

In [ ]:
#plot coordinates of stars in secondary peak for 5500 K stars at 200 pc
number = 0
plt.figure()
for i in range(len(G_K_200)):
    if (G_K_200[i,2] == 5500 and G_K_200[i,0] >= 1.7 and G_K_200[i,0] <= 2.0):
        number = number +1
        plt.scatter(G_K_200[i,3],G_K_200[i,4])
plt.xlabel('RA')
plt.ylabel('Dec')
plt.show()
print(number)

In [ ]:
#Calculate average G-Mag for all temperatures 
from matplotlib.backends.backend_pdf import PdfPages
pp = PdfPages('Template_Differences.pdf')

diff150 = np.zeros([len(temps2),len(wvl)])
diff200 = np.zeros([len(temps2),len(wvl)])
diff_std150 = np.zeros([len(temps2),len(wvl)])
diff_std200 = np.zeros([len(temps2),len(wvl)])
for i in range(0,len(temps2)): 
    for j in range(len(wvl)):
        diff150[i,j] = temp_mags150[i,j] - temp_mags100[i,j]
        diff200[i,j] = temp_mags200[i,j] - temp_mags100[i,j]
        diff_std150[i,j] = math.sqrt(math.pow(temp_std150[i,j],2) + math.pow(temp_std100[i,j],2))
        diff_std200[i,j] = math.sqrt(math.pow(temp_std200[i,j],2) + math.pow(temp_std100[i,j],2))
    num_star = num_stars[i+1]
    plt.figure()
    plt.errorbar(wvl, diff150[i,:],yerr = diff_std150[i,:],linestyle = '-', marker = 'o',label = '150 pc')
    plt.errorbar(wvl, diff200[i,:],yerr = diff_std200[i,:],linestyle = '-', marker = 'o',label = '200 pc')
    plt.axhline(linestyle = ':', color = 'k')
    plt.ylim(ymin=-0.5,ymax=0.5)
    plt.title('Difference in Template for '+ str(temps2[i]) + 'K (' + str(num_star) + ' stars)' )
    plt.xlabel('Wavelength (nm)')
    plt.ylabel('Difference in G-Mag')
    plt.legend()
    pp.savefig()
pp.close()


In [ ]:
#Plot G-K and G-B over each temperature bin for 100 pc, 150 pc and 200 pc

pp2 = PdfPages('Temperature_Differences.pdf')
#Plot of G-K
plt.errorbar(temps2, diff150[:,4],yerr = diff_std150[:,4],linestyle = '-', marker = 'o',label = '150 pc')
plt.errorbar(temps2, diff200[:,4],yerr = diff_std200[:,4],linestyle = '-', marker = 'o',label = '200 pc')
plt.axhline(linestyle = ':', color = 'k')
plt.title('Difference in G-K for each Temperature Category' )
plt.xlabel('Temperature Category (K)')
plt.ylabel('Difference in G-K')
plt.legend()
pp2.savefig()
plt.show()

#Plot of G-B
plt.figure()
plt.errorbar(temps2, diff150[:,1],yerr = diff_std150[:,1],linestyle = '-', marker = 'o',label = '150 pc')
plt.errorbar(temps2, diff200[:,1],yerr = diff_std200[:,1],linestyle = '-', marker = 'o',label = '200 pc')
plt.axhline(linestyle = ':', color = 'k')
plt.title('Difference in G-B for each Temperature Category' )
plt.xlabel('Temperature Category (K)')
plt.ylabel('Difference in G-B')
plt.legend()
pp2.savefig()
plt.show()

pp2.close()

In [ ]:
#Plot relative SED by distance 

plt.figure()
for i in range(2,len(temps2)): 
    plt.errorbar(wvl,diff150[i,:],yerr = diff_std150[i,:],linestyle = '-', marker = 'o',label = str(temps2[i]))
    plt.title('Difference in Template for 150 pc')
    plt.xlabel('Wavelength (nm)')
    plt.ylabel('Difference in G-Mag')
    plt.legend()

plt.figure()
for i in range(2,len(temps2)): 
    plt.errorbar(wvl,diff200[i,:],yerr = diff_std200[i,:],linestyle = '-', marker = 'o',label = str(temps2[i]))
    plt.title('Difference in Template for 200 pc')
    plt.xlabel('Wavelength (nm)')
    plt.ylabel('Difference in G-Mag')
    plt.legend()

mean_diff150 = np.zeros(len(wvl))
mean_diff_err150 = np.zeros(len(wvl))
weight_mean150 = np.zeros(len(wvl))
for i in range(len(wvl)):
    mean_diff150[i] = np.mean(diff150[:,i])
    mean_diff_err150[i] = np.std(diff150[:,i])
    weightsum = 0;
    weights = 0
    for j in range (len(temps2)):
        weightsum = weightsum+stars150[j]*diff150[j,i]
        weights = weights + stars150[j]
    weight_mean150[i] = weightsum/weights    
plt.figure()
plt.errorbar(wvl, mean_diff150, yerr = mean_diff_err150, linestyle = '-', marker = 'o')
plt.plot(wvl,weight_mean150,linestyle = '-', marker = 'o')
plt.title('Difference in Template for 150 pc')
plt.xlabel('Wavelength (nm)')
plt.ylabel('Difference in G-Mag')

mean_diff200 = np.zeros(len(wvl))
mean_diff_err200 = np.zeros(len(wvl))
weight_mean200 = np.zeros(len(wvl))
for i in range(len(wvl)):
    mean_diff200[i] = np.mean(diff200[:,i])
    mean_diff_err200[i] = np.std(diff200[:,i])
    weightsum = 0;
    weights = 0
    for j in range (len(temps2)):
        weightsum = weightsum+stars200[j]*diff200[j,i]
        weights = weights + stars200[j]
    weight_mean200[i] = weightsum/weights    
plt.figure()
plt.errorbar(wvl, mean_diff200, yerr = mean_diff_err200, linestyle = '-', marker = 'o')
plt.plot(wvl,weight_mean200,linestyle = '-', marker = 'o')
plt.title('Difference in Template for 200 pc')
plt.xlabel('Wavelength (nm)')
plt.ylabel('Difference in G-Mag')
plt.show()


In [ ]:
#Extinction 
wvl4 = [420/1000, 532/1000, 1235/1000, 1662/1000, 2159/1000]
wvl3 = np.array([1.65, 1.25, 0.55, 0.44])
Av = np.array([0.176, 0.282, 1.00, 1.31])
Color_Ex = (np.ones(len(Av)) - Av)/10

plt.figure()
plt.plot(wvl3,Color_Ex,linestyle = '-', marker = 'o',label = 'Expected')
plt.xlabel('Wavelength (micrometers)')
plt.ylabel('Expected Color Excess')
plt.plot(wvl4,weight_mean150,linestyle = '-', marker = 'o',label = '150 pc')
plt.plot(wvl4,weight_mean200,linestyle = '-', marker = 'o', label  = '200 pc')
plt.legend()
plt.show()


In [ ]:
#Plot G-K and G-B over each temperature bin for 100 pc, 150 pc and 200 pc

temps3 = [4000,4500,5000,5500,6000,6500,7000,7500,8000,8500,9000,9500]
print(diff150)

pp2 = PdfPages('Temperature_Differences.pdf')
#Plot of G-K
plt.errorbar(temps2, diff150[:,4],yerr = diff_std150[:,4],linestyle = '-', marker = 'o',label = '150 pc')
plt.errorbar(temps2, diff200[:,4],yerr = diff_std200[:,4],linestyle = '-', marker = 'o',label = '200 pc')
plt.axhline(linestyle = ':', color = 'k')
plt.title('Difference in G-K for each Temperature Category' )
plt.xlabel('Temperature Category (K)')
plt.ylabel('Difference in G-K')
plt.legend()
pp2.savefig()
plt.show()

#Plot of G-B
plt.figure()
plt.errorbar(temps2, diff150[:,1],yerr = diff_std150[:,1],linestyle = '-', marker = 'o',label = '150 pc')
plt.errorbar(temps2, diff200[:,1],yerr = diff_std200[:,1],linestyle = '-', marker = 'o',label = '200 pc')
plt.axhline(linestyle = ':', color = 'k')
plt.title('Difference in G-B for each Temperature Category' )
plt.xlabel('Temperature Category (K)')
plt.ylabel('Difference in G-B')
plt.legend()
pp2.savefig()
plt.show()

pp2.close()

In [ ]:
#Plot G-K and G-B over each temperature bin for 100 pc, 150 pc and 200 pc

pp3 = PdfPages('Temperature_Differences2.pdf')
#Plot of G-K
plt.errorbar(temps2, diff150[:,4],yerr = diff_std150[:,4],linestyle = '-', marker = 'o',label = '150 pc')
plt.errorbar(temps2, diff150[:,4],yerr = temp_std[:,4],linestyle = '-', marker = 'o',label = '150 pc (Template Error)')
plt.errorbar(temps2, diff200[:,4],yerr = diff_std200[:,4],linestyle = '-', marker = 'o',label = '200 pc')
plt.errorbar(temps2, diff200[:,4],yerr = temp_std[:,4],linestyle = '-', marker = 'o',label = '200 pc (Template Error)')
plt.axhline(linestyle = ':', color = 'k')
plt.title('Difference in G-K for each Temperature Category' )
plt.xlabel('Temperature Category (K)')
plt.ylabel('Difference in G-K')
plt.legend()
pp3.savefig()
plt.show()

#Plot of G-B
plt.figure()
plt.errorbar(temps2, diff150[:,1],yerr = diff_std150[:,1],linestyle = '-', marker = 'o',label = '150 pc')
plt.errorbar(temps2, diff150[:,1],yerr = temp_std[:,1],linestyle = '-', marker = 'o',label = '150 pc (Template Error)')
plt.errorbar(temps2, diff200[:,1],yerr = diff_std200[:,1],linestyle = '-', marker = 'o',label = '200 pc')
plt.errorbar(temps2, diff200[:,1],yerr = temp_std[:,1],linestyle = '-', marker = 'o',label = '200 pc (Template Error)')
plt.axhline(linestyle = ':', color = 'k')
plt.title('Difference in G-B for each Temperature Category' )
plt.xlabel('Temperature Category (K)')
plt.ylabel('Difference in G-B')
plt.legend()
pp3.savefig()
plt.show()

pp3.close()

In [ ]:
#assign stars of 6000 K a distance category
distance_cat2 = np.zeros(len(Dist))
for i in range(len(temp_cat)):
    if (temp_cat[i] == 6000):
        for j in range(20,200,20):
            if (j == 20):
                if (Dist[i] <= j):
                    distance_cat2[i] = j
            if (Dist[i] <= j and Dist[i] > j-20):
                distance_cat2[i] = j 
print(distance_cat2)          

In [ ]:
#Determine average value of G-Mag for each distance category 
distances = list(range(20,200,20))
for i in range(20,200,20):
    G_B_array = ma.array([])
    G_V_array = ma.array([])
    G_J_array = ma.array([])
    G_H_array = ma.array([])
    G_K_array = ma.array([])
    for j in range(len(distance_cat2)):
        if (distance_cat2[j] == i):
            G_V_array = np.append(G_V_array,Gmags[j]-Vmags[j])
            G_B_array = np.append(G_B_array,Gmags[i]-Bmags[i])
            G_J_array = np.append(G_J_array,Gmags[i]-Jmags[i])
            G_H_array = np.append(G_H_array,Gmags[i]-Hmags[i])
            G_K_array = np.append(G_K_array,Gmags[i]-Kmags[i])
    array = [np.mean(G_B_array), np.mean(G_V_array), np.mean(G_J_array), np.mean(G_H_array), np.mean(G_K_array)]
    err  = [np.std(G_B_array), np.std(G_V_array), np.std(G_J_array), np.std(G_H_array), np.std(G_K_array)]
    if (i == 20):
        dist_array = array
        err_array = err
    if (i != 20):
        dist_array = np.vstack((dist_array,array))
        err_array = np.vstack((err_array,err))
print(dist_array)
print(err_array)
for i in range(0,4):
    plt.figure()
    plt.errorbar(distances, dist_array[:,i], yerr = err_array[:,i], linestyle = '-', marker = 'o')
    plt.xlabel('Distance Category (pc)')
    if i == 0: plt.ylabel('G-B')
    if i == 1: plt.ylabel('G-V')
    if i == 2: plt.ylabel('G-J')
    if i == 3: plt.ylabel('G-H')
    if i == 4: plt.ylabel('G-K')
    plt.title('Average value for G-Mag for each Distance Category (6000 K)')
   
     
        
        

In [ ]:
#Plot G-Mag of all stars minus G-Mag for distance stars
all_diff100 = np.zeros([len(temps2),len(wvl)])
all_diff150 = np.zeros([len(temps2),len(wvl)])
all_diff200 = np.zeros([len(temps2),len(wvl)])
all_diff_std100 = np.zeros([len(temps2),len(wvl)])
all_diff_std150 = np.zeros([len(temps2),len(wvl)])
all_diff_std200 = np.zeros([len(temps2),len(wvl)])
for i in range(0,len(temps2)): 
    for j in range(len(wvl)):
        all_diff100[i,j] = temp_mags[i,j] - temp_mags100[i,j]
        all_diff150[i,j] = temp_mags[i,j] - temp_mags150[i,j]
        all_diff200[i,j] = temp_mags[i,j] - temp_mags200[i,j]
        all_diff_std100[i,j] = math.sqrt(math.pow(temp_std[i,j],2) + math.pow(temp_std100[i,j],2))
        all_diff_std150[i,j] = math.sqrt(math.pow(temp_std[i,j],2) + math.pow(temp_std150[i,j],2))
        all_diff_std200[i,j] = math.sqrt(math.pow(temp_std[i,j],2) + math.pow(temp_std200[i,j],2))
    num_star = num_stars[i+1]
    plt.figure()
    plt.errorbar(wvl, all_diff100[i,:],yerr = all_diff_std100[i,:],linestyle = '-', marker = 'o',label = '100 pc')
    plt.errorbar(wvl, all_diff150[i,:],yerr = all_diff_std150[i,:],linestyle = '-', marker = 'o',label = '150 pc')
    plt.errorbar(wvl, all_diff200[i,:],yerr = all_diff_std200[i,:],linestyle = '-', marker = 'o',label = '200 pc')
    plt.axhline(linestyle = ':', color = 'k')
    plt.ylim(ymin=-0.5,ymax=0.5)
    plt.title('Difference in Template for '+ str(temps2[i]) + 'K (' + str(num_star) + ' stars)' )
    plt.xlabel('Wavelength (nm)')
    plt.ylabel('Difference in G-Mag')
    plt.legend()




In [ ]:
#Plot relative SED by distance 
plt.figure()
for i in range(2,len(temps2)): 
    plt.errorbar(wvl,all_diff100[i,:],yerr = all_diff_std100[i,:],linestyle = '-', marker = 'o',label = str(temps2[i]))
    plt.title('Difference in Template for 100 pc')
    plt.xlabel('Wavelength (nm)')
    plt.ylabel('Difference in G-Mag')
    plt.legend()

plt.figure()
for i in range(2,len(temps2)): 
    plt.errorbar(wvl,all_diff150[i,:],yerr = all_diff_std150[i,:],linestyle = '-', marker = 'o',label = str(temps2[i]))
    plt.title('Difference in Template for 150 pc')
    plt.xlabel('Wavelength (nm)')
    plt.ylabel('Difference in G-Mag')
    plt.legend()

plt.figure()
for i in range(2,len(temps2)): 
    plt.errorbar(wvl,all_diff200[i,:],yerr = all_diff_std200[i,:],linestyle = '-', marker = 'o',label = str(temps2[i]))
    plt.title('Difference in Template for 200 pc')
    plt.xlabel('Wavelength (nm)')
    plt.ylabel('Difference in G-Mag')
    plt.legend()